<a href="https://colab.research.google.com/github/Kadaytes/HW_PY/blob/main/DZ_5_VNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [32]:
import re

In [33]:
with open('train_data.txt', 'r', encoding='utf-8') as f:
  text = f.read()
text = text.replace('\ufeff', '') # убираем первый невидимый символ
text = re.sub(r'[^А-я ]', '', text) # убираем все недопустимые символы

In [35]:
num_characters = 34 #33 буквы + пробел

In [36]:
import keras
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=num_characters, char_level=True)

tokenizer.fit_on_texts(text)

In [37]:
print(tokenizer.word_index)

{' ': 1, 'о': 2, 'е': 3, 'и': 4, 'н': 5, 'с': 6, 'т': 7, 'а': 8, 'в': 9, 'л': 10, 'ы': 11, 'к': 12, 'д': 13, 'м': 14, 'р': 15, 'г': 16, 'п': 17, 'ь': 18, 'у': 19, 'з': 20, 'х': 21, 'б': 22, 'й': 23, 'ч': 24, 'ю': 25, 'я': 26, 'ф': 27, 'ц': 28, 'ж': 29, 'э': 30}


In [38]:
inp_chars = 3
data = tokenizer.texts_to_matrix(text)
data

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [39]:
n = data.shape[0]-inp_chars
n #размер обучающего множества

619

In [40]:
import numpy as np
X = np.array([data[i:i+inp_chars, :] for i in range(n)])
Y = data[inp_chars:] #предсказание следующего символа


In [41]:
from keras.layers import *
from keras.models import Sequential
model = Sequential()
model.add(Input((inp_chars, num_characters)))
model.add(SimpleRNN(500, activation='tanh'))
model.add(Dense(num_characters, activation='softmax'))
model.summary()
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
history = model.fit(X, Y, batch_size=32, epochs=100)


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_3 (SimpleRNN)    (None, 500)               267500    
                                                                 
 dense_3 (Dense)             (None, 34)                17034     
                                                                 
Total params: 284534 (1.09 MB)
Trainable params: 284534 (1.09 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/100
20/20 [==============================] - 1s 9ms/step - loss: 3.2598 - accuracy: 0.1179
Epoch 2/100
20/20 [==============================] - 0s 11ms/step - loss: 2.7188 - accuracy: 0.2310
Epoch 3/100
20/20 [==============================] - 0s 9ms/step - loss: 2.4759 - accuracy: 0.2892
Epoch 4/100
20/20 [==============================] - 0s 9ms/step - loss: 2.2958 - accuracy: 0.3344
Epoch 5/100
20/20

In [42]:
def buildPhrase(inp_str, str_len = 50):
  for i in range(str_len):
    x = []
    for j in range(i, i+inp_chars):
      x.append(tokenizer.texts_to_matrix(inp_str[j])) # преобразуем символы в One-Hot-encoding
    x = np.array(x)
    inp = x.reshape(1, inp_chars, num_characters)

    pred = model.predict( inp ) # предсказываем OHE четвертого символа
    d = tokenizer.index_word[pred.argmax(axis=1)[0]] # получаем ответ в символьном представлении

    inp_str += d # дописываем строку

  return inp_str


In [43]:
res = buildPhrase("достиг")
print(res)

1/1 [==============================] - 0s 31ms/step
достигти аа нттиыйе   мндпозуобитароеловньвеы ык в омосб


**Слова**

In [44]:
with open('train_data.txt', 'r', encoding='utf-8') as f:
  texts = f.read()
texts = texts.replace('\ufeff', '') # убираем первый невидимый символ
maxWordsCount = 1000
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!–"—#$%&amp;()*+,-./:;<=>?@[\\]^_`{|}~\t\n\r«»',
lower=True, split=' ', char_level=False)
tokenizer.fit_on_texts([texts])
dist = list(tokenizer.word_counts.items())
print(dist[:10])


[('исследование', 1), ('функций', 1), ('мозга', 3), ('человека', 1), ('и', 5), ('попытки', 1), ('воссоздать', 1), ('его', 1), ('сложные', 1), ('процессы', 1)]


In [45]:
data = tokenizer.texts_to_sequences([texts])
res = keras.utils.to_categorical(data[0], num_classes=maxWordsCount)
print( res.shape )

(84, 1000)


In [46]:
inp_words = 3
n = res.shape[0]-inp_words
X = np.array([res[i:i+inp_words, :] for i in range(n)])
Y = res[inp_words:]
model = Sequential()
model.add(Input((inp_words, maxWordsCount)))
model.add(SimpleRNN(128, activation='tanh'))
model.add(Dense(maxWordsCount, activation='softmax'))
model.summary()




Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_4 (SimpleRNN)    (None, 128)               144512    
                                                                 
 dense_4 (Dense)             (None, 1000)              129000    
                                                                 
Total params: 273512 (1.04 MB)
Trainable params: 273512 (1.04 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [47]:

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
history = model.fit(X, Y, batch_size=32, epochs=50)


Epoch 1/50
3/3 [==============================] - 1s 9ms/step - loss: 6.9067 - accuracy: 0.0000e+00
Epoch 2/50
3/3 [==============================] - 0s 9ms/step - loss: 6.8600 - accuracy: 0.0370
Epoch 3/50
3/3 [==============================] - 0s 11ms/step - loss: 6.8157 - accuracy: 0.2840
Epoch 4/50
3/3 [==============================] - 0s 9ms/step - loss: 6.7693 - accuracy: 0.6173
Epoch 5/50
3/3 [==============================] - 0s 9ms/step - loss: 6.7157 - accuracy: 0.8395
Epoch 6/50
3/3 [==============================] - 0s 9ms/step - loss: 6.6504 - accuracy: 0.9012
Epoch 7/50
3/3 [==============================] - 0s 9ms/step - loss: 6.5662 - accuracy: 0.8519
Epoch 8/50
3/3 [==============================] - 0s 9ms/step - loss: 6.4512 - accuracy: 0.7531
Epoch 9/50
3/3 [==============================] - 0s 9ms/step - loss: 6.2820 - accuracy: 0.5185
Epoch 10/50
3/3 [==============================] - 0s 11ms/step - loss: 6.0343 - accuracy: 0.2346
Epoch 11/50
3/3 [================

In [48]:
def buildPhrase(texts, str_len = 20):
  res = texts
  data = tokenizer.texts_to_sequences([texts])[0]
  for i in range(str_len):
      x = to_categorical(data[i: i+inp_words], num_classes=maxWordsCount) # преобразуем в One-Hot-encoding
  inp = x.reshape(1, inp_words, maxWordsCount)

  pred = model.predict( inp ) # предсказываем OHE четвертого символа
  indx = pred.argmax(axis=1)[0]
  data.append(indx)

  res += " " + tokenizer.index_word[indx] # дописываем строку

  return res

In [51]:
res = buildPhrase("для более глубокого")
print(res)

Разобрался в программе и попробовал на разных текстах. Ошибку в проге по словам так и не нашел.
Итого: Увеличение объема текста обучения улучшает немного результат.
Хотя в общем то пока всё печально ...